In [ ]:
import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv
import os
# dotenv.load_dotenv(".env.local")
dotenv.load_dotenv(".env.kpzdev")

from mrp_functions.base_tasks import TaskBase
import re


np.__version__

tb = TaskBase()
part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(os.environ['SIMULATION'])

mrp_or_calendar_frame = tb.query_mrp_order_calendars(part_group, contract_code)

if mrp_or_calendar_frame is None or len(mrp_or_calendar_frame) == 0:
    raise ValueError(f"MRP Ordering Calendar Not Found for Part Group: {part_group} and Contract Code: {contract_code}")

mrp_or_calendar_frame

In [ ]:
# Target From: W4-09/2022. Target To: W2-10/2022. Buffer: W1-11/2022~W2-11/2022
target_from = 'W4-09/2022'
target_to = 'W2-10/2022'
buffer_from = 'W1-11/2022'
buffer_to = 'W2-11/2022'
re.split("\D", target_from)
_, target_from_week, target_from_month_no, target_from_year = re.split("\D", target_from)
_, target_to_week, target_to_month_no, target_to_year = re.split("\D", target_to)
if buffer_from is not None and len(buffer_from) > 0 and buffer_to is not None and len(buffer_to) > 0:
    _, buffer_from_week, buffer_from_month_no, buffer_from_year = re.split("\D", buffer_from)
    _, buffer_to_week, buffer_to_month_no, buffer_to_year = re.split("\D", buffer_to)

    target_date = tb.query_date_from_week_definition(target_from_week, target_from_month_no, target_from_year,
                                                     buffer_to_week, buffer_to_month_no, buffer_to_year)
else:
    target_date = tb.query_date_from_week_definition(target_from_week, target_from_month_no, target_from_year,
                                                     target_to_week, target_to_month_no, target_to_year)

print(f"[{contract_code}_{part_group}_{production_plan_id}_{mrp_run_date}] "
      f"Target From: {target_from}. Target To: {target_to}. Buffer: {buffer_from}~{buffer_to}")
if target_date is None or len(target_date) == 0:
    raise ValueError(f"Target Plan is invalid")

print(f"[{contract_code}_{part_group}_{production_plan_id}_{mrp_run_date}] "
      f"Target Date: {target_date[0]} To: {target_date[-1]}")

In [ ]:
target_from, target_to, buffer_from, buffer_to = mrp_or_calendar_frame.loc[0, ["target_plan_from", "target_plan_to", "buffer_span_from", "buffer_span_to"]].values
print(target_from)
print(target_to)
print(buffer_from)
print(buffer_to)

_, target_from_week, target_from_month_no, target_from_year = re.split("\D", target_from)
_, target_to_week, target_to_month_no, target_to_year = re.split("\D", target_to)
if buffer_from is not None and len(buffer_from) > 0 and buffer_to is not None and len(buffer_to) > 0:
    _, buffer_from_week, buffer_from_month_no, buffer_from_year = re.split("\D", buffer_from)
    _, buffer_to_week, buffer_to_month_no, buffer_to_year = re.split("\D", buffer_to)

    target_date = tb.query_date_from_week_definition(target_from_week, target_from_month_no, target_from_year,
                                                     buffer_to_week, buffer_to_month_no, buffer_to_year)
else:
    target_date = tb.query_date_from_week_definition(target_from_week, target_from_month_no, target_from_year,
                                                     target_to_week, target_to_month_no, target_to_year)
if len(target_date) > 1:
    print(target_date[0])
    print(target_date[-1])

In [ ]:
print(target_date[0])
print(target_date[-1])

print(f"[{contract_code}_{part_group}_{production_plan_id}_{mrp_run_date}] "
      f"Check shortage within production plan !")
shortage_parts = tb.query_shortage_parts(mrp_run_date, production_plan_id, part_group, target_date[0], target_date[-1])
shortage_parts

In [ ]:
shortage_parts = shortage_parts.loc[~shortage_parts.duplicated(keep='first')]

In [ ]:
shortage_parts.groupby(["part_code", "part_color_code", "part_group", "supplier_code",
                                                 "minimum_order_quantity", "standard_box_quantity",
                                                 "part_quantity_in_box", "import_id", "plant_code"
                                                 ]).agg({"quantity": sum})


In [ ]:

order_list = shortage_parts[(shortage_parts['production_date'] >= target_date.iloc[0].date) &
                                    (shortage_parts['production_date'] <= target_date.iloc[-1].date)] \
            .groupby(["part_code", "part_color_code", "part_group", "supplier_code",
                      "minimum_order_quantity", "standard_box_quantity", "part_quantity_in_box", "import_id",
                      "plant_code"
                      ]).agg({"quantity": sum})

order_list.to_dict("records")


In [ ]:
order_list.reset_index(inplace=True)

order_list[['eta']] = mrp_or_calendar_frame.iloc[0].eta
order_list[['contract_code']] = mrp_or_calendar_frame.iloc[0].contract_code
order_list.to_dict("records")


In [ ]:
result = order_list.copy(deep=True)

result['quantity'] = result.apply(
                lambda row: np.multiply(row['part_quantity_in_box'],
                                        np.ceil(np.divide(np.max((np.abs(row['quantity']), row['minimum_order_quantity'])),
                                                          row['part_quantity_in_box']))), axis=1)

# result[['quantity']] = np.multiply(np.ceil(
#     (np.max((0 - result[['quantity']], result[['minimum_order_quantity']]))) /
#     np.multiply(result[['standard_box_quantity']], result[['part_quantity_in_box']])
# ), result[['standard_box_quantity']], result[['part_quantity_in_box']])
result.to_dict("records")

